In [1]:
#https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925

# Turkish Stemmer Test

In [57]:
from snowballstemmer import TurkishStemmer
turkStem=TurkishStemmer()
turkStem.stemWord("gelmişti")

'gelmiş'

# Import Libraries and Load Data

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import warnings
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')

#Import Data
data = pd.read_excel('/content/drive/MyDrive/turkish_topic_modelling.xlsx')
print(data.head(10))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
       Class                                           Document
0  astronomi  NGC 5713 Başak takımyıldızı bölgesinde bulunan...
1  astronomi  Birçok katalogda sarmal gökada olarak sınıflan...
2    hükümet  Corina Casanova , İsviçre Federal Şansölyesidir .
3       yasa  Casanova , İsviçre Federal Yüksek Mahkemesi es...
4    hükümet       Corina Casanova bir federal parlementerdir .
5      bölge  Casanova , Hristiyan Demokrat Halk Partisi üye...
6    hükümet  İsviçre Dışişleri Bakanlığı , İsviçre federal ...
7    hükümet  İsviçre'nin dış ilişkilerini sürdürmekle görev...
8      bölge  Gilgit Baltistan veya ( Pakistan Kuzey Bölgele...
9      bölge  72496 km alan kaplamakta ve oldukça dağlık bir...


Checking data

In [73]:
df=data['Document']
df

0         NGC 5713 Başak takımyıldızı bölgesinde bulunan...
1         Birçok katalogda sarmal gökada olarak sınıflan...
2         Corina Casanova , İsviçre Federal Şansölyesidir .
3         Casanova , İsviçre Federal Yüksek Mahkemesi es...
4              Corina Casanova bir federal parlementerdir .
                                ...                        
647804    24 yaşındaki 1.86 metre boyundaki Senegalli go...
647805    Senegalli forvet Sion takımıyla 2009 2010 sezo...
647806    Sahadan 2 2'lik skorla ayrılan Sion , Fenerbah...
647807    IP2bölge kullanıcılara aynı adla bir yazılım d...
647808    IP2bölge ziyaretçilerin IP adresi ( ülke , böl...
Name: Document, Length: 647809, dtype: object

In [75]:
#!pip install gensim
import gensim
from gensim.utils import simple_preprocess
#from gensim.parsing.preprocessing import STOPWORDS           #Does not support Turkish yet.
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [76]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
print(WordNetLemmatizer().lemmatize('gelmişti', pos = 'v')) # unfortunately, not working for Turkish

gelmişti


In [77]:
import pandas as pd
from snowballstemmer import TurkishStemmer
stemmer=TurkishStemmer()
original_words = ['Başarılı', 'insanlar', 'adamlar', 'öldüler', 'içindekiler','kapısındaki', 'yiyecekler,', 'çıkaranlar', 
           'lahanalar', 'takımların','sırası', 'futbolcuların', 'yedikleri']
singles = [stemmer.stemWord(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,Başarılı,Başarıl
1,insanlar,in
2,adamlar,adam
3,öldüler,öl
4,içindekiler,içindeki
5,kapısındaki,kapıs
6,"yiyecekler,","yiyecekler,"
7,çıkaranlar,çıkaran
8,lahanalar,lahana
9,takımların,tak


• As it is seen, stemmer is not perfect but it works

# Stemming and Tokenizing Functions

In [78]:
def lemmatize_stemming(text):
    return stemmer.stemWord(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stop_word_list and len(token) > 3:
            result.append(lemmatize_stemming(token))
     
    return result

In [79]:
doc_sample = 'Bu disk çok fazla kez hata verdi. Mümkünse bunu başka bir tane ile değiştirmek istiyorum.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['Bu', 'disk', 'çok', 'fazla', 'kez', 'hata', 'verdi.', 'Mümkünse', 'bunu', 'başka', 'bir', 'tane', 'ile', 'değiştirmek', 'istiyorum.']


Tokenized and lemmatized document: 
['disk', 'faz', 'ha', 'ver', 'mümk', 'p', 'başka', 'tane', 'değiştirmek', 'istiyor']


• Some of words are misstemmed ("hata" --> ha etc.)

In [80]:
processed_docs = []

for doc in data['Document']:
    processed_docs.append(preprocess(doc))

In [81]:
print(processed_docs[:2])

[['başak', 'takımyıldız', 'bölge', 'buluna', 'tuhaf', 'asimetrik', 'göka'], ['birçok', 'katalogu', 'sarmal', 'göka', 'olarak', 'sınıflandırıla', 'sarmal', 'gökada', 'oldukça', 'farklı']]


# Crating Dictionary for LDA

In [82]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [83]:
count = 0                         #Testing if it is created
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 asimetrik
1 başak
2 buluna
3 bölge
4 göka
5 takımyıldız
6 tuhaf
7 birçok
8 farklı
9 gökada
10 katalogu


In [84]:
'''
Remove very rare and very common words: (Probably not working for Turkish but I tried)
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [85]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [86]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 11 ("olarak") appears 1 time.
Word 99 ("leopardı") appears 1 time.
Word 102 ("snow") appears 1 time.
Word 133 ("intel") appears 1 time.
Word 138 ("tabanlı") appears 1 time.
Word 145 ("ağustos") appears 1 time.
Word 146 ("bilgisayar") appears 1 time.
Word 147 ("macintosh") appears 1 time.
Word 148 ("tarih") appears 1 time.
Word 149 ("yayınla") appears 1 time.
Word 150 ("yükseltme") appears 1 time.


# LDA Model

In [88]:

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 20, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [89]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.018*"dil" + 0.017*"olarak" + 0.013*"özellik" + 0.010*"olduk" + 0.010*"kaynak" + 0.008*"yuna" + 0.008*"ola" + 0.007*"diğer" + 0.007*"olmas" + 0.007*"örnek"


Topic: 1 
Words: 0.014*"klip" + 0.012*"video" + 0.011*"çekil" + 0.011*"çek" + 0.011*"karar" + 0.011*"dikkat" + 0.010*"taraf" + 0.009*"be" + 0.009*"çizgi" + 0.009*"charles"


Topic: 2 
Words: 0.035*"sıra" + 0.020*"liste" + 0.020*"dünya" + 0.018*"üçüncü" + 0.015*"milyo" + 0.014*"büyük" + 0.013*"ol" + 0.011*"ulaş" + 0.011*"faz" + 0.011*"kadar"


Topic: 3 
Words: 0.041*"başka" + 0.020*"görev" + 0.020*"olarak" + 0.019*"g" + 0.016*"ay" + 0.016*"seçil" + 0.014*"başkanlık" + 0.011*"ol" + 0.011*"genel" + 0.010*"yap"


Topic: 4 
Words: 0.063*"tarih" + 0.029*"türk" + 0.022*"türki" + 0.018*"taraf" + 0.016*"stanbul" + 0.015*"olarak" + 0.013*"kurul" + 0.012*"ocak" + 0.012*"mayıs" + 0.011*"hazira"


Topic: 5 
Words: 0.029*"birlik" + 0.028*"devlet" + 0.023*"ülke" + 0.019*"avrup" + 0.017*"amerik" + 0.015*"birleşik" + 0.014*"taraf

# Unseen Text Testing

In [98]:
unseen_document = "Onun müzik kulağı yeteneklerin de ötesindedir. Keman gitar piyano ve daha birçok enstrümanın üstadıdır dersek yeridir"
print(unseen_document)

Onun müzik kulağı yeteneklerin de ötesindedir. Keman gitar piyano ve daha birçok enstrümanın üstadıdır dersek yeridir


In [99]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.25007668137550354	 Topic: 0.056*"grup" + 0.044*"alp" + 0.043*"şarkı" + 0.030*"müzik" + 0.018*"albüm"
Score: 0.20905724167823792	 Topic: 0.024*"olduk" + 0.018*"kendi" + 0.016*"ola" + 0.013*"sonra" + 0.010*"çocuk"
Score: 0.18945811688899994	 Topic: 0.050*"olarak" + 0.012*"ola" + 0.010*"kabul" + 0.010*"farklı" + 0.010*"genellik"
Score: 0.07775552570819855	 Topic: 0.025*"güney" + 0.021*"dok" + 0.021*"kuzey" + 0.018*"or" + 0.017*"bölge"
Score: 0.07722388207912445	 Topic: 0.063*"tarih" + 0.029*"türk" + 0.022*"türki" + 0.018*"taraf" + 0.016*"stanbul"
Score: 0.07500000298023224	 Topic: 0.014*"klip" + 0.012*"video" + 0.011*"çekil" + 0.011*"çek" + 0.011*"karar"
Score: 0.07500000298023224	 Topic: 0.033*"kiş" + 0.023*"gör" + 0.021*"nüfus" + 0.014*"göl" + 0.012*"yüzölç"


Results are not bad. Even if there are nonesense words (because of stemming) it works generally.




